In [6]:
import os
import subprocess
import shutil
import sys
from varseek.utils import create_identity_t2g

varseek_directory = os.path.dirname(os.path.abspath(""))

conda_env_path = os.path.dirname(os.path.dirname(shutil.which("kb")))  # to get kb path
operating_system = "linux" if sys.platform.startswith("linux") else "darwin/m1"

kallisto = f"{conda_env_path}/lib/python3.10/site-packages/kb_python/bins/{operating_system}/kallisto/kallisto"  # or kallisto_k64
bustools = f"{conda_env_path}/lib/python3.10/site-packages/kb_python/bins/{operating_system}/bustools/bustools"

In [9]:
ref_fa = f"{varseek_directory}/tests/kb_files/ref_test.fa"
read1_fq = f"{varseek_directory}/tests/kb_files/reads1_test.fq"
read2_fq = f"{varseek_directory}/tests/kb_files/reads2_test.fq"
test_index = f"{varseek_directory}/tests/kb_files/index_test.idx"
test_t2g = f"{varseek_directory}/tests/kb_files/t2g_test.txt"
kb_count_out_test = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_with_num"
kb_count_out_test2 = f"{varseek_directory}/tests/kb_files/test_kb_count_out_March2025_without_num"

rnaseq_fastq_files_final = [read1_fq, read2_fq]
os.makedirs(kb_count_out_test, exist_ok=True)

if not os.path.exists(test_index):
    kb_ref_command = ["kb", "ref", "--workflow", "custom", "-t", str(8), "-i", test_index, "--d-list", "None", ref_fa]
    subprocess.run(kb_ref_command, check=True)

if not os.path.exists(test_t2g):
    create_identity_t2g(ref_fa, test_t2g)

if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_test, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

if not os.path.exists(kb_count_out_test2) or len(os.listdir(kb_count_out_test2)) == 0:
    kb_count_command = ["kb", "count", "-t", str(2), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--h5ad", "--parity", "single", "-o", kb_count_out_test2, "--kallisto", kallisto, "--bustools", bustools] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

# kb_count_out_test = f"{varseek_directory}/tests/kb_files/test_kb_count_out_paired"
# if not os.path.exists(kb_count_out_test) or len(os.listdir(kb_count_out_test)) == 0:
#     kb_count_command = ["kb", "count", "-t", str(8), "-i", test_index, "-g", test_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "paired", "-o", kb_count_out_test] + rnaseq_fastq_files_final
#     subprocess.run(kb_count_command, check=True)

[2025-02-28 01:12:50,949]    INFO [count] Using index /Users/joeyrich/Desktop/local/varseek/tests/kb_files/index_test.idx to generate BUS file to /Users/joeyrich/Desktop/local/varseek/tests/kb_files/test_kb_count_out_March2025_with_num from
[2025-02-28 01:12:50,950]    INFO [count]         /Users/joeyrich/Desktop/local/varseek/tests/kb_files/reads1_test.fq
[2025-02-28 01:12:50,950]    INFO [count]         /Users/joeyrich/Desktop/local/varseek/tests/kb_files/reads2_test.fq
[2025-02-28 01:12:52,065]    INFO [count] Sorting BUS file /Users/joeyrich/Desktop/local/varseek/tests/kb_files/test_kb_count_out_March2025_with_num/output.bus to /Users/joeyrich/Desktop/local/varseek/tests/kb_files/test_kb_count_out_March2025_with_num/tmp/output.s.bus
[2025-02-28 01:12:53,693]    INFO [count] Inspecting BUS file /Users/joeyrich/Desktop/local/varseek/tests/kb_files/test_kb_count_out_March2025_with_num/tmp/output.s.bus
[2025-02-28 01:12:54,807]    INFO [count] Generating count matrix /Users/joeyrich/De

In [10]:
bus_text_file = os.path.join(kb_count_out_test, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

bus_text_file = os.path.join(kb_count_out_test2, "bus_text.txt")
bus_file = os.path.join(kb_count_out_test2, "output.bus")
bustools_text_command = f"{bustools} text -o {bus_text_file} -f {bus_file}"
subprocess.run(bustools_text_command, check=True, shell=True)

Read in 6 BUS records
Read in 6 BUS records


CompletedProcess(args='/Users/joeyrich/miniconda3/envs/varseek/lib/python3.10/site-packages/kb_python/bins/darwin/m1/bustools/bustools text -o /Users/joeyrich/Desktop/local/varseek/tests/kb_files/test_kb_count_out_March2025_without_num/bus_text.txt -f /Users/joeyrich/Desktop/local/varseek/tests/kb_files/test_kb_count_out_March2025_without_num/output.bus', returncode=0)

In [ ]:
import importlib
import varseek.utils.seq_utils
importlib.reload(varseek.utils.seq_utils)
from varseek.utils.seq_utils import make_bus_df
bus_df = make_bus_df(kallisto_out = kb_count_out_test, fastq_file_list = rnaseq_fastq_files_final, t2g_file = test_t2g, mm = False, union = False, technology = "bulk", bustools = bustools)